## UFRN - EEC2006 - TOPICOS ESPECIAIS F
## Segundo trabalho 

Componentes:
* **20171021275 - Fabio Fonseca de Oliveira**
* **2016102462 - Júlio César Melo Gomes de Oliveira**
* **20171021201 - Tiago Fernandes de Miranda**

Notebook com solução do segundo trabalho proposto na disciplina. A seguir breve descrição de cada solução bem como suas informações específicas.

#### 1) Mapa dinâmico da população das cidades do Nordeste do Brasil

Neste trabalho temos por objetivo levantar a população de cada cidade da região nordeste do Brasil e apresenta-las de forma dinâmica em mapa de cores baseado na população de cada cidade.

Link do vídeo explicativo: https://www.youtube.com/watch?v=sZr13jO46MY

#### 2) Mapa dinâmico do tempo de espera de solicitações do serviço UBER dividido pelos bairros de Natal

Neste trabalho faz-se necessário disponibilizar um mapa interativo que forneça o tempo médio de espera de usuários do UBER dividido por bairros da cidade de Natal. 

Link do vídeo explicativo: https://www.youtube.com/watch?v=w8aromvCWTc

---

*Importação dos pacotes necessários para o trabalho*

In [1]:
import folium
import json
import pandas as pd
from branca.colormap import linear
from branca.utilities import split_six
import numpy as np
from shapely.geometry import Polygon
from shapely.geometry import Point
from numpy import random
import os
import datetime as date
from uber_rides.session import Session
from uber_rides.client import UberRidesClient
import csv
import time
from IPython.display import IFrame

### 1) Mapa dinâmico da população das cidades do Nordeste do Brasil

*Carregando dataset csv da população*

In [2]:
data2017 = pd.read_csv('population_2017.csv')

*Tratamento dos nomes das colunas do dataset*

In [3]:
data2017.columns = [cols.replace(' ', '_') for cols in data2017.columns]

In [4]:
data2017.head()

,UF,COD._UF,COD._MUNIC,NOME_DO_MUNICÍPIO,POPULAÇÃO_ESTIMADA
0,RO,11.0,15.0,Alta Floresta D'Oeste,25437.0
1,RO,11.0,23.0,Ariquemes,107345.0
2,RO,11.0,31.0,Cabixi,6224.0
3,RO,11.0,49.0,Cacoal,88507.0
4,RO,11.0,56.0,Cerejeiras,17934.0


*Carregamento do arquivo geojson da região nordeste do Brasil*

In [5]:
geo_json_name_file = 'geojs-NE-mun.json'
geo_json_data = json.load(open(geo_json_name_file,encoding='latin-1'))

*Seleção do subconjunto contendo os dados populacionais das cidades dos estados do nordeste do Brasil*

In [6]:
dataNE = data2017[(data2017['UF'] == 'RN') | (data2017['UF'] == 'BA') | (data2017['UF'] == 'AL')
                 | (data2017['UF'] == 'CE') | (data2017['UF'] == 'MA') | (data2017['UF'] == 'PB')
                 | (data2017['UF'] == 'PE') | (data2017['UF'] == 'PI') | (data2017['UF'] == 'SE')
                 ]

*Reordenação dos dados selecionados com base no nome das cidades*

In [7]:
dataNE = dataNE.sort_values('NOME_DO_MUNICÍPIO')
dataNE.head()

,UF,COD._UF,COD._MUNIC,NOME_DO_MUNICÍPIO,POPULAÇÃO_ESTIMADA
891,CE,23.0,101.0,Abaiara,11605.0
1828,BA,29.0,207.0,Abaré,20189.0
1827,BA,29.0,108.0,Abaíra,9199.0
1465,PE,26.0,54.0,Abreu e Lima,99364.0
1829,BA,29.0,306.0,Acajutiba,15727.0


*Configurando o index do dataset atual e obtendo apenas os dados populacionais*

In [8]:
population_dict = dataNE.set_index('NOME_DO_MUNICÍPIO')['POPULAÇÃO_ESTIMADA']
population_dict.head()

NOME_DO_MUNICÍPIO
Abaiara         11605.0
Abaré           20189.0
Abaíra           9199.0
Abreu e Lima    99364.0
Acajutiba       15727.0
Name: POPULAÇÃO_ESTIMADA, dtype: float64

*Criando mapa com centro na região nordeste*

In [9]:
m = folium.Map(
    location=[-9.6837525, -40.7405724],
    zoom_start=6,
    tiles='Stamen Terrain'
)

*Configurando a legenda de cores do mapa*

In [10]:
population_min = dataNE['POPULAÇÃO_ESTIMADA'].min()
population_max = dataNE['POPULAÇÃO_ESTIMADA'].max()
threshold_scale = np.linspace(population_min,population_max,6,dtype=int).tolist()

*Configurando o mapa e salvando em arquivo externo*

In [11]:
m.choropleth(
    geo_data=geo_json_data,
    data=dataNE,
    columns=['NOME_DO_MUNICÍPIO', 'POPULAÇÃO_ESTIMADA'],
    key_on='feature.properties.description',
    fill_color='YlOrRd',
    legend_name='Population estimation (2017)',
    highlight=True,
    threshold_scale = threshold_scale
)

m.save('pop_nordeste_map.html')

*Carregando arquivo externo resultante e disponibilizando em um frame*

In [12]:
IFrame('pop_nordeste_map.html', width=975, height=815)

### 2) Mapa dinâmico do tempo de espera de solicitações do serviço UBER dividido pelos bairros de Natal

*Configura conexão com API do UBER através de token*

In [13]:
session = Session(server_token='ZZgDb4jQCbM3loUFQN8-QSpQubTKiX6qp14o-k63')
client = UberRidesClient(session)

*Carrega os arquivos geojson de Natal*

In [14]:
geo_json_natal = json.load(open('natal.geojson',encoding='UTF-8'))

*Definição de duas funções: uma que cria pontos aleatórios dentro dos poligonos associados aos bairros
 e outra que retorna uma coluna de uma matriz*

In [15]:
def generate_random(number, polygon, polys_nhood):
    list_of_points = []
    minx, miny, maxx, maxy = polygon.bounds
    counter = 0
    while counter < number:
        x = random.uniform(minx, maxx)
        y = random.uniform(miny, maxy)
        pnt = Point(x, y)
        if polygon.contains(pnt):
            list_of_points.append([x,y,polys_nhood])
            counter += 1
    return list_of_points

In [16]:
def column(matrix,i):
    return [row[i] for row in matrix]

*Código responsável por gerar pontos aleatórios dentro de cada bairro*

In [17]:
number_of_points = 500
neigh_rand_points = []
count = 0
for feature in geo_json_natal['features']:
    neighborhood = feature['properties']['name']
    geom = feature['geometry']['coordinates']
    polygon = Polygon(geom[0])
    points = generate_random(number_of_points,polygon, neighborhood)
    for i,value in enumerate(points):
        log, lat, name = value           
        neigh_rand_points.append([count,i,log,lat,name,0,0])
        count = count + 1
            
len(neigh_rand_points)

18000

*Organiza o nome das colunas do dataframe recém criado, seguindo o formato utilizado pela API do UBER*

In [18]:
fieldnames  = ['key', 'idx', 'logintude', 'latitude', 'neighborhood', 'time(s)', 'money(R$)']
data = {fieldnames[0]: column(neigh_rand_points,0), 
        fieldnames[1]: column(neigh_rand_points,1), 
        fieldnames[2]: column(neigh_rand_points,2),
        fieldnames[3]: column(neigh_rand_points,3),
        fieldnames[4]: column(neigh_rand_points,4),
        fieldnames[5]: column(neigh_rand_points,5),
        fieldnames[6]: column(neigh_rand_points,6)}
df = pd.DataFrame(data, columns=fieldnames)

*Script de coleta do tempo de espera para cada ponto gerado aleatoriamente*

In [19]:
coletar_dados = 0
if coletar_dados:
    n_count = 0
    print(date.datetime.now(), ' - Iniciando coleta dados')
    for index, row  in df.iterrows():
        latitude = df.loc[index,'latitude']
        longitude = df.loc[index,'logintude']
        neighborhood_name =  df.loc[index,'neighborhood']
        wait_time = client.get_pickup_time_estimates(start_latitude = latitude, start_longitude = longitude)                  
     
        if(len(wait_time.json.get('times'))==0):
            print(date.datetime.now(), ' - Valor nulo obtido: [',index,'=>',neighborhood_name,' (',latitude,',',longitude,')]')
        else:
            df.loc[index,'time(s)'] = wait_time.json.get('times')[0]['estimate']     
        
        n_count = n_count + 1
        time.sleep(2)
        if(n_count==50):
            print(date.datetime.now(), ' - Salvando dados parciais no arquivo externo (',index,'/',df.shape[0],')')
            filename = 'neigh_rand_points.csv'
            df.to_csv('neigh_rand_points.csv', encoding = 'utf-8', index=False)
            n_count = 0        

    print(date.datetime.now(), ' - Coleta de dados finalizada')

*Lendo arquivo CSV resultante da coleta dos dados para processamento do mapa*

In [20]:
df = pd.read_csv('dados_uber_bkp_03112017.csv',encoding = 'utf-8')

*Obtendo nome dos bairros a partir do CSV resultante da coleta*

In [21]:
name_neigh = []
name_neigh.append([p for p in df['neighborhood'].unique()])
name_neigh = name_neigh[0]
print(name_neigh)

['Pitimbu', 'Planalto', 'Ponta Negra', 'Neópolis', 'Capim Macio', 'Lagoa Azul', 'Pajuçara', 'Lagoa Seca', 'Barro Vermelho', 'Candelária', 'Praia do Meio', 'Rocas', 'Santos Reis', 'Redinha', 'Salinas', 'Igapó', 'Nossa Senhora da Apresentação', 'Potengi', 'Ribeira', 'Cidade Alta', 'Alecrim', 'Nordeste', 'Quintas', 'Bom Pastor', 'Dix-Sept Rosado', 'Nossa Senhora de Nazaré', 'Lagoa Nova', 'Mãe Luiza', 'Nova Descoberta', 'Tirol', 'Petrópolis', 'Areia Preta', 'Cidade Nova', 'Cidade da Esperança', 'Felipe Camarão', 'Guarapes']


*Preparando dataframe que armazenará o tempo médio de espera para cada bairro*

In [22]:
fieldnames  = ['neighborhood', 'mean_time(s)']
zero = np.zeros(len(name_neigh), dtype=np.int).tolist()
data = {fieldnames[0]: name_neigh, 
        fieldnames[1]: zero}
neigh_data = pd.DataFrame(data, columns=fieldnames)
neigh_data.head()

,neighborhood,mean_time(s)
0,Pitimbu,0
1,Planalto,0
2,Ponta Negra,0
3,Neópolis,0
4,Capim Macio,0


*Para cada bairro é feito o calculo do tempo médio de espera, em segundos*

In [23]:
for i,name in neigh_data.iterrows():
    listN = df['time(s)'].where(df['neighborhood'] == name['neighborhood']).dropna()    
    neigh_data.loc[i,'mean_time(s)'] = sum(listN)/len(listN)
neigh_data.head()

,neighborhood,mean_time(s)
0,Pitimbu,263.16
1,Planalto,563.16
2,Ponta Negra,395.16
3,Neópolis,336.36
4,Capim Macio,356.04


*Configurando legenda de cores do mapa*

In [24]:
# threshold_scale = np.linspace(neigh_data['mean_time(s)'].min(),
#                               neigh_data['mean_time(s)'].max(), 10, dtype=float).tolist()
threshold_scale =  split_six(neigh_data['mean_time(s)'])

*Criação do mapa e carregamento do arquivo geojson de Natal no mapa*

In [25]:
m1 = folium.Map(
    location=[-5.826592, -35.212558],
    zoom_start=11,
    tiles='Stamen Terrain'
)

*Configuração do mapa e salvando em arquivo externo*

In [26]:
m1.choropleth(
    geo_data=geo_json_natal,
    data=neigh_data,
    columns=['neighborhood', 'mean_time(s)'],
    key_on='feature.properties.name',
    fill_color='YlGnBu',
    legend_name='Time estimation in seconds (2017)',
    highlight=True,
    threshold_scale = threshold_scale
)

m1.save('wait_time_uber_natal.html')

*Carregando arquivo externo resultante e disponibilizando em um frame*

In [27]:
IFrame('wait_time_uber_natal.html', width=975, height=815)